# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, concat, col, lit
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import RegexTokenizer, VectorAssembler

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 

In [9]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [4]:
stack_overflow_data = 'Train_onetag_small.json'

In [5]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

In [18]:
df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- oneTag: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- BodyLength: integer (nullable = true)



### Build Body Length Feature

In [11]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [16]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [17]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [28]:
df.where(df.Id == 1112).show()

+--------------------+----+--------------------+--------------------+------+--------------------+----------+
|                Body|  Id|                Tags|               Title|oneTag|               words|BodyLength|
+--------------------+----+--------------------+--------------------+------+--------------------+----------+
|<p>I submitted my...|1112|iphone app-store ...|iPhone app releas...|iphone|[p, i, submitted,...|        63|
+--------------------+----+--------------------+--------------------+------+--------------------+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [31]:
df = df.withColumn("Desc", concat(col("Title"), lit(" "), col("Body")))

In [32]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words2", pattern="\\W")
df = regexTokenizer.transform(df)
df = df.withColumn("DescLength", body_length(df.words2))

In [34]:
df.where(df.Id == 5123).select("DescLength").collect()

[Row(DescLength=135)]

# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [36]:
assembler2 = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler2.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [ ]:
# TODO: write your code to answer question 3

# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [ ]:
# TODO: write your code to answer question 4

# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [ ]:
# TODO: write your code to answer question 5